# Categorizing urban density

Exploration of an academic study of urban structure and density described in the June 2014 article, ["From Jurisdictional to Functional Analysis of Urban Cores & Suburbs"](http://www.newgeography.com/content/004349-from-jurisdictional-functional-analysis-urban-cores-suburbs) in [new geography](http://www.newgeography.com/). 

## Categories

- Urban (pre-auto urban core): density > 2,900 sq. km
- Auto suburban, early: median house built 1946 to 1979, density < 2,900 sq. km and density > 100 sq. km
- Auto suburban, later: median house built after 1979, density < 2,900 sq. km and density > 100 sq. km
- Auto exurban: all others


In [1]:
import pandas as pd, numpy as np

## Collect U.S. Census data

Census data from the 2013 US Census American Community Survey (ACS), 5-year estimates.

Created from the "zip code tabulation area" (ZCTA) [TIGER/Line® with Selected Demographic and Economic Data product in Geodatabase format](http://www.census.gov/geo/maps-data/data/tiger-data.html). This particular version of the ACS is used for the folowing reasons:

1. 5-year estimates are the most accurate data outside of the decennial census [as explained here](http://www.census.gov/programs-surveys/acs/guidance/estimates.html).
1. 2013 is the most recent data set with 5-year estimates
1. TIGER/Line® gives you the geographic boundaries of the zip codes so you can perform spatial analyses
1. This data set is smaller than the full Census, but still has the important income, education, race, age and occupation demographics we want to use.

If you want to do this yourself, [this article](https://developer.ibm.com/clouddataservices/2015/09/08/census-open-data-on-ibm-cloud/) explains how to get a CSV out of that format.


### Get zip code areas from Census

In [2]:
geo_df = pd.read_csv('http://opendata.mybluemix.net/uscensus/areas.csv.gz',
                    usecols=['GEOID10','ALAND10'], dtype={"GEOID10": np.str})
geo_df['GEOID'] = "86000US" + geo_df['GEOID10']
geo_df = geo_df.set_index('GEOID')
geo_df.head()

,GEOID10,ALAND10
GEOID,,
86000US43451,43451,63411475
86000US43452,43452,121783680
86000US43456,43456,9389360
86000US43457,43457,48035540
86000US43458,43458,2573816


### Get population from Census

In [3]:
pop_df = pd.read_csv('http://opendata.mybluemix.net/uscensus/x01_age_sex.csv.gz',
                     usecols=['GEOID','B01001e1'], dtype={"GEOID": np.str})
pop_df = pop_df.set_index('GEOID')
pop_df.head()

,B01001e1
GEOID,
86000US01001,17245
86000US01002,29266
86000US01003,11032
86000US01005,5356
86000US01007,14673


### Get housing age from Census

In [4]:
housing_df = pd.read_csv('http://opendata.mybluemix.net/uscensus/x25_housing_characteristics.csv.gz',
                     usecols=['GEOID','B25035e1'], dtype={"GEOID": np.str})
housing_df = housing_df.set_index('GEOID')
housing_df.sample(5)

,B25035e1
GEOID,
86000US48360,1990
86000US84026,1977
86000US10706,1939
86000US28606,1981
86000US68454,1951


### Join Census data into one DataFrame with nice names

In [5]:
urban_df = geo_df.join(pop_df)
urban_df = urban_df.join(housing_df)

In [6]:
urban_df.columns = ['ZIP','AREAMSQ','Population','MEDYRBUILT']
urban_df.sample(5)

,ZIP,AREAMSQ,Population,MEDYRBUILT
GEOID,,,,
86000US24060,24060,343525684,53541,1980
86000US15610,15610,99894576,3650,1977
86000US14618,14618,25069000,22110,1958
86000US79738,79738,833166609,301,1974
86000US44437,44437,10211388,4127,1956


## Density calculation
persons per square km = persons / (area in square meters / 1,000,000)
persons per hectare = persons / (area in square meters / 10,000)

### Compute population density as persons per square kilometer

In [7]:
urban_df['POPPERKMSQ'] = urban_df['Population'] / (urban_df['AREAMSQ']/1000000)
urban_df.sample(4)

,ZIP,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ
GEOID,,,,,
86000US61537,61537,152917002,2871,1956,18.774891
86000US20765,20765,1207946,465,1939,384.950983
86000US66854,66854,261101579,781,1958,2.991173
86000US63113,63113,6574098,12239,1939,1861.700267


### Group population density into 4 categories

In [8]:
urban_df['CAT'] = 'EXURBAN'
urban_df['CAT'][(urban_df['POPPERKMSQ'] >= 2900)] = 'URBAN'
urban_df['CAT'][(urban_df['POPPERKMSQ'] < 2900) & (urban_df['POPPERKMSQ'] >= 100) & (urban_df['MEDYRBUILT'] < 1980) & (urban_df['MEDYRBUILT'] >= 1946)] = 'SUBURBANEARLY'
urban_df['CAT'][(urban_df['POPPERKMSQ'] < 2900) & (urban_df['POPPERKMSQ'] >= 100) & (urban_df['MEDYRBUILT'] >= 1980)] = 'SUBURBANLATE'
urban_df.describe()

/Users/rajrsingh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/rajrsingh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/rajrsingh/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ
count,3.298900e+04,32989.000000,32045.000000,32989.000000
mean,2.250019e+08,9443.177453,1971.068529,487.703913
std,6.575933e+08,13858.010530,15.606758,1912.093435
min,5.094000e+03,0.000000,1939.000000,0.000000
25%,2.351832e+07,719.000000,1961.000000,7.754449
50%,9.322068e+07,2781.000000,1974.000000,30.194351
75%,2.304373e+08,12830.000000,1982.000000,249.247358
max,3.478591e+10,114734.000000,2011.000000,71226.281507


In [9]:
# look at a few records to do a quick sanity check
urban_df.sample(30)

,ZIP,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ,CAT
GEOID,,,,,,
86000US62675,62675,282900573,6065,1973,21.438627,EXURBAN
86000US39063,39063,251294074,3477,1977,13.836379,EXURBAN
86000US63530,63530,258105985,871,1975,3.374583,EXURBAN
86000US71969,71969,85824437,315,1966,3.670283,EXURBAN
86000US71406,71406,20585313,367,1982,17.828245,EXURBAN
86000US48228,48228,22299440,52191,1952,2340.462361,SUBURBANEARLY
86000US62326,62326,196757473,2267,1959,11.521799,EXURBAN
86000US54806,54806,686984914,11496,1950,16.733992,EXURBAN
86000US03457,03457,59525163,695,1968,11.675735,EXURBAN


## Mapping Urbanity

In [10]:
!pip install --user cartodb
# Set up cartodb module for use
from cartodb import CartoDBAPIKey, CartoDBException
# Learn how to use the CartoDB SQL API here:
# http://docs.cartodb.com/cartodb-platform/sql-api/making-calls/
API_KEY = 'yourapikeyhere'
CARTODB_ACCOUNT = 'youraccountnamehere'
cl = CartoDBAPIKey(API_KEY, CARTODB_ACCOUNT)

### Update data table in CartoDB

In [11]:
# get all non-exurban zips
notex_df = urban_df[ urban_df['CAT'] != 'EXURBAN' ]
notex_df.head()

,ZIP,AREAMSQ,Population,MEDYRBUILT,POPPERKMSQ,CAT
GEOID,,,,,,
86000US43452,43452,121783680,13659,1974,112.157885,SUBURBANEARLY
86000US43458,43458,2573816,312,1960,121.220787,SUBURBANEARLY
86000US43460,43460,7158543,6334,1959,884.816924,SUBURBANEARLY
86000US43465,43465,28559486,5267,1974,184.422087,SUBURBANEARLY
86000US43468,43468,1861424,403,1957,216.500915,SUBURBANEARLY


In [12]:
# generate PostgreSQL INSERT statements for CartoDB
insertsql = ''
for index, row in notex_df.iterrows(): 
    insertsql += "INSERT INTO urbanity(zipcode,category) VALUES ('" + index + "', '" + row['CAT'] + "');"

In [13]:
# send SQL INSERT statements to CartoDB
try:
    print(cl.sql('delete from urbanity'))
    print(cl.sql(insertsql))
except CartoDBException as e:
    print("some error ocurred", e)

{u'fields': {}, u'rows': [], u'total_rows': 10002, u'time': 0.079}
{u'fields': {}, u'rows': [], u'total_rows': 1, u'time': 53.493}


### Urbanity Map!

In [15]:
%%javascript
element.append("<link rel='stylesheet' href='http://libs.cartocdn.com/cartodb.js/v3/3.15/themes/css/cartodb.css' />")
element.append("<div id='map' style='height:500px;width:900px;padding:0;margin:0'></div>");

require.config({
  paths: {
      cartodblib: 'http://libs.cartocdn.com/cartodb.js/v3/3.15/cartodb'
  }
});

var main = function() {
  cartodb.createVis('map', 'https://ibmanalytics.cartodb.com/u/ibm/api/v2/viz/3a4e9e24-b3f6-11e5-a4c7-0e31c9be1b51/viz.json', {
      shareable: true,title: true,description: true,search: true,tiles_loader: true,
      center_lat: 40, center_lon: -100, zoom: 3
  })
  .done(function(vis, layers) {
    // layer 0 is the base layer, layer 1 is cartodb layer
    // setInteraction is disabled by default
    layers[1].setInteraction(true);
    layers[1].on('featureOver', function(e, latlng, pos, data) {
      cartodb.log.log(e, latlng, pos, data);
    });
    
    // var map = vis.getNativeMap(); // get the native map to work with it
    
  })
  .error(function(err) {
    console.log(err);
  });
}

require(['cartodblib'], main);

<IPython.core.display.Javascript object>